In [ ]:
!pip install scikit-learn
import sklearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 109.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [scikit-learn] [scikit-learn]


In [ ]:
probe_dataset_entities = [
    # Writers
    {"name": "Stephen King", "profession": "writer", "nationality": "American"},
    {"name": "Haruki Murakami", "profession": "writer", "nationality": "Japanese"},
    {"name": "Gabriel García Márquez", "profession": "writer", "nationality": "Colombian"},
    {"name": "Charles Dickens", "profession": "writer", "nationality": "British"},
    {"name": "Virginia Woolf", "profession": "writer", "nationality": "British"},
    {"name": "Jane Austen", "profession": "writer", "nationality": "British"},
    
    # Physicists/Scientists
    {"name": "Albert Einstein", "profession": "physicist", "nationality": "German"},
    {"name": "Stephen Hawking", "profession": "physicist", "nationality": "British"},
    {"name": "Marie Curie", "profession": "scientist", "nationality": "Polish"},
    {"name": "Rosalind Franklin", "profession": "scientist", "nationality": "British"},
    {"name": "Nikola Tesla", "profession": "inventor", "nationality": "Serbian"},
    {"name": "Thomas Edison", "profession": "inventor", "nationality": "American"},
    
    # Artists/Painters
    {"name": "Vincent van Gogh", "profession": "painter", "nationality": "Dutch"},
    {"name": "Frida Kahlo", "profession": "painter", "nationality": "Mexican"},
    {"name": "Claude Monet", "profession": "painter", "nationality": "French"},
    {"name": "Pablo Picasso", "profession": "painter", "nationality": "Spanish"},
    {"name": "Andy Warhol", "profession": "artist", "nationality": "American"},
    {"name": "Banksy", "profession": "artist", "nationality": "British"},
    
    # Musicians/Composers
    {"name": "Mozart", "profession": "composer", "nationality": "Austrian"},
    {"name": "Tchaikovsky", "profession": "composer", "nationality": "Russian"},
    {"name": "Beethoven", "profession": "composer", "nationality": "German"},
    {"name": "Chopin", "profession": "composer", "nationality": "Polish"},
    
    # Actors/Actresses
    {"name": "Meryl Streep", "profession": "actress", "nationality": "American"},
    {"name": "Sophia Loren", "profession": "actress", "nationality": "Italian"},
    {"name": "Audrey Hepburn", "profession": "actress", "nationality": "British"},
    {"name": "Marilyn Monroe", "profession": "actress", "nationality": "American"},
    
    # Athletes
    {"name": "Pelé", "profession": "footballer", "nationality": "Brazilian"},
    {"name": "Diego Maradona", "profession": "footballer", "nationality": "Argentinian"},
    {"name": "Cristiano Ronaldo", "profession": "footballer", "nationality": "Portuguese"},
    {"name": "Lionel Messi", "profession": "footballer", "nationality": "Argentinian"},
    
    # Philosophers
    {"name": "Socrates", "profession": "philosopher", "nationality": "Greek"},
    {"name": "Confucius", "profession": "philosopher", "nationality": "Chinese"},
    {"name": "Immanuel Kant", "profession": "philosopher", "nationality": "German"},
    {"name": "René Descartes", "profession": "philosopher", "nationality": "French"},
    
    # Psychologists
    {"name": "Sigmund Freud", "profession": "psychologist", "nationality": "Austrian"},
    {"name": "Carl Jung", "profession": "psychologist", "nationality": "Swiss"},
    
    # Directors
    {"name": "Alfred Hitchcock", "profession": "director", "nationality": "British"},
    {"name": "Akira Kurosawa", "profession": "director", "nationality": "Japanese"},
    
    # Architects
    {"name": "Frank Lloyd Wright", "profession": "architect", "nationality": "American"},
    {"name": "Le Corbusier", "profession": "architect", "nationality": "Swiss"},
]

print(f"Total entities in probe dataset: {len(probe_dataset_entities)}")


Total entities in probe dataset: 40


In [ ]:
# Let's analyze the distribution of attributes
profession_counts = {}
nationality_counts = {}

for entity in probe_dataset_entities:
    profession = entity["profession"]
    nationality = entity["nationality"]
    
    profession_counts[profession] = profession_counts.get(profession, 0) + 1
    nationality_counts[nationality] = nationality_counts.get(nationality, 0) + 1

print("\nProfession distribution:")
for profession, count in sorted(profession_counts.items()):
    print(f"  {profession}: {count}")

print("\nNationality distribution:")
for nationality, count in sorted(nationality_counts.items()):
    print(f"  {nationality}: {count}")

# Create label mappings for our probes
unique_professions = sorted(list(profession_counts.keys()))
unique_nationalities = sorted(list(nationality_counts.keys()))

profession_to_id = {profession: i for i, profession in enumerate(unique_professions)}
nationality_to_id = {nationality: i for i, nationality in enumerate(unique_nationalities)}

print(f"\nWe have {len(unique_professions)} unique professions and {len(unique_nationalities)} unique nationalities")
print(f"This gives us a robust dataset for training linear probes!")


Profession distribution:
  actress: 4
  architect: 2
  artist: 2
  composer: 4
  director: 2
  footballer: 4
  inventor: 2
  painter: 4
  philosopher: 4
  physicist: 2
  psychologist: 2
  scientist: 2
  writer: 6

Nationality distribution:
  American: 6
  Argentinian: 2
  Austrian: 2
  Brazilian: 1
  British: 8
  Chinese: 1
  Colombian: 1
  Dutch: 1
  French: 2
  German: 3
  Greek: 1
  Italian: 1
  Japanese: 2
  Mexican: 1
  Polish: 2
  Portuguese: 1
  Russian: 1
  Serbian: 1
  Spanish: 1
  Swiss: 2

We have 13 unique professions and 20 unique nationalities
This gives us a robust dataset for training linear probes!


In [ ]:
### Step 2: Extract Activations for Linear Probing

# We'll use a simple prompt format that just presents the entity name
# This is cleaner than the complex BiAssociation prompt for probing purposes
probe_prompt_template = "The famous person {name} is known for being a professional"

# Prepare all prompts for our probe dataset
probe_prompts = []
probe_labels_profession = []
probe_labels_nationality = []
probe_entity_names = []

for entity in probe_dataset_entities:
    prompt = probe_prompt_template.format(name=entity["name"])
    probe_prompts.append(prompt)
    
    # Convert labels to integers for our classifiers
    probe_labels_profession.append(profession_to_id[entity["profession"]])
    probe_labels_nationality.append(nationality_to_id[entity["nationality"]])
    probe_entity_names.append(entity["name"])

print(f"Prepared {len(probe_prompts)} prompts for probing")
print(f"Example prompt: '{probe_prompts[0]}'")
print(f"Corresponding profession label: {probe_labels_profession[0]} ({unique_professions[probe_labels_profession[0]]})")
print(f"Corresponding nationality label: {probe_labels_nationality[0]} ({unique_nationalities[probe_labels_nationality[0]]})")

# Tokenize all the prompts to understand their structure
probe_inputs = prepare_input(probe_prompts, tokenizer=mt.tokenizer, return_offsets_mapping=True)
offset_mappings = probe_inputs.pop("offset_mapping")

# Let's analyze the token structure of our prompts
print(f"\nToken analysis for first prompt:")
tokens = probe_inputs.input_ids[0]
decoded_tokens = [mt.tokenizer.decode([tok]) for tok in tokens]
print(f"Tokens: {decoded_tokens}")

# Find where entity names are located in each prompt
entity_token_ranges = []
for i, (entity_name, offset_mapping) in enumerate(zip(probe_entity_names, offset_mappings)):
    try:
        # Find the token range that corresponds to the entity name
        entity_range = find_token_range(
            string=probe_prompts[i],
            substring=entity_name,
            offset_mapping=offset_mapping,
            tokenizer=mt.tokenizer,
            occurrence=0  # First (and only) occurrence
        )
        entity_token_ranges.append(entity_range)
    except Exception as e:
        print(f"Warning: Could not find entity range for {entity_name}: {e}")
        # Fallback: assume entity is in the middle tokens
        entity_token_ranges.append((3, 5))  # Rough estimate

print(f"Entity token ranges (first few): {entity_token_ranges[:5]}")

# Define which layers we want to probe (using same layers as patchscoping for comparison)
probe_layer_indices = layer_indices  # Reuse the same layers: [5, 15, 25, 31, 33, 35, 37, 39, 41, 45, 55, 65, 75]
print(f"Will probe at layers: {probe_layer_indices}")


In [ ]:
### Step 3: Extract Activations from Model

# We'll extract activations at multiple positions for each entity:
# 1. At entity token positions (like in patchscoping)
# 2. At the last token position 
# This allows us to see how representations differ by position

print("Extracting activations from model...")
print("This may take a few minutes due to the large model size...")

# Define locations where we want to extract activations
probe_locations = []

# Add all layer-position combinations we're interested in
for layer_idx in probe_layer_indices:
    layer_name = mt.layer_name_format.format(layer_idx)
    
    # We'll probe at multiple positions:
    # 1. Last token position (like in patchscoping analysis)
    probe_locations.append((layer_name, -1))
    
    # 2. A few key positions that typically contain entity information
    # (we'll average over entity token positions for each entity)
    for pos in [3, 4, 5, 6, 7]:  # Common positions where entity names appear
        probe_locations.append((layer_name, pos))

print(f"Total locations to probe: {len(probe_locations)}")

# Extract activations using the same approach as in the patchscoping analysis
probe_activations = get_hs(
    mt=mt,
    input=TokenizerOutput(data=probe_inputs),
    locations=probe_locations
)

print(f"Successfully extracted activations!")
print(f"Shape of activations for first location: {probe_activations[probe_locations[0]].shape}")

# The activations tensor has shape [batch_size, hidden_dim] where:
# - batch_size = number of entities in our probe dataset
# - hidden_dim = model's hidden dimension (typically 4096+ for large models)
activation_dim = probe_activations[probe_locations[0]].shape[1]
print(f"Model hidden dimension: {activation_dim}")


In [ ]:
### Step 4: Train Linear Probes with Cross-Validation

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
from collections import defaultdict

# We'll use a pipeline that standardizes features before applying logistic regression
# This is important because activation magnitudes can vary significantly across layers
def create_probe_pipeline():
    return Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', LogisticRegression(
            max_iter=1000,      # Ensure convergence
            class_weight='balanced',  # Handle class imbalance
            random_state=42     # Reproducible results
        ))
    ])

# We'll use stratified k-fold cross-validation to get robust accuracy estimates
cv_folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("Training linear probes with cross-validation...")
print("This will train probes for both profession and nationality classification")
print("at each layer and position combination...")

# Store results in organized dictionaries
# Structure: probe_results[attribute][layer_idx][position] = [cv_scores]
probe_results = {
    'profession': defaultdict(lambda: defaultdict(list)),
    'nationality': defaultdict(lambda: defaultdict(list))
}

# Track progress
total_probes = len(probe_layer_indices) * 6 * 2  # layers * positions * attributes
current_probe = 0

for layer_idx in probe_layer_indices:
    layer_name = mt.layer_name_format.format(layer_idx)
    
    print(f"\nTraining probes for layer {layer_idx}...")
    
    # Probe at different positions
    positions_to_probe = [-1, 3, 4, 5, 6, 7]  # Last token + entity positions
    
    for position in positions_to_probe:
        location = (layer_name, position)
        
        if location not in probe_activations:
            print(f"  Warning: No activations found for {location}, skipping...")
            continue
            
        # Get activations for this layer-position combination
        activations = probe_activations[location]  # Shape: [n_entities, hidden_dim]
        
        # Convert to numpy for sklearn compatibility
        X = activations.cpu().numpy() if hasattr(activations, 'cpu') else activations
        
        # Train profession probe
        current_probe += 1
        print(f"  Position {position}: Training profession probe ({current_probe}/{total_probes})...")
        
        y_profession = np.array(probe_labels_profession)
        profession_pipeline = create_probe_pipeline()
        
        # Perform cross-validation
        profession_cv_scores = cross_val_score(
            profession_pipeline, X, y_profession, 
            cv=cv_folds, scoring='accuracy'
        )
        
        probe_results['profession'][layer_idx][position] = profession_cv_scores
        
        # Train nationality probe
        current_probe += 1
        print(f"  Position {position}: Training nationality probe ({current_probe}/{total_probes})...")
        
        y_nationality = np.array(probe_labels_nationality)
        nationality_pipeline = create_probe_pipeline()
        
        nationality_cv_scores = cross_val_score(
            nationality_pipeline, X, y_nationality,
            cv=cv_folds, scoring='accuracy'
        )
        
        probe_results['nationality'][layer_idx][position] = nationality_cv_scores
        
        # Print intermediate results
        prof_mean = profession_cv_scores.mean()
        prof_std = profession_cv_scores.std()
        nat_mean = nationality_cv_scores.mean()
        nat_std = nationality_cv_scores.std()
        
        print(f"    Profession accuracy: {prof_mean:.3f} ± {prof_std:.3f}")
        print(f"    Nationality accuracy: {nat_mean:.3f} ± {nat_std:.3f}")

print(f"\nProbe training complete! Trained {current_probe} probes total.")


In [ ]:
### Step 5: Analyze Probe Results and Create Comprehensive Visualizations

# First, let's compute summary statistics for easier analysis
probe_summary = {
    'profession': {},
    'nationality': {}
}

# Calculate mean accuracy and confidence intervals for each layer-position combination
for attribute in ['profession', 'nationality']:
    for layer_idx in probe_layer_indices:
        probe_summary[attribute][layer_idx] = {}
        for position in [-1, 3, 4, 5, 6, 7]:
            if position in probe_results[attribute][layer_idx]:
                cv_scores = probe_results[attribute][layer_idx][position]
                mean_acc = np.mean(cv_scores)
                std_acc = np.std(cv_scores)
                # 95% confidence interval
                                 ci_lower = mean_acc - 1.96 * std_acc / (len(cv_scores) ** 0.5)
                 ci_upper = mean_acc + 1.96 * std_acc / (len(cv_scores) ** 0.5)
                
                probe_summary[attribute][layer_idx][position] = {
                    'mean': mean_acc,
                    'std': std_acc,
                    'ci_lower': ci_lower,
                    'ci_upper': ci_upper,
                    'scores': cv_scores
                }

# Print summary table of best performing layer-position combinations
print("=== LINEAR PROBE RESULTS SUMMARY ===\n")

for attribute in ['profession', 'nationality']:
    print(f"{attribute.upper()} CLASSIFICATION:")
    print("-" * 50)
    
    # Find best performing combinations
    best_results = []
    for layer_idx in probe_layer_indices:
        for position in [-1, 3, 4, 5, 6, 7]:
            if position in probe_summary[attribute][layer_idx]:
                result = probe_summary[attribute][layer_idx][position]
                best_results.append((layer_idx, position, result['mean'], result['std']))
    
    # Sort by accuracy
    best_results.sort(key=lambda x: x[2], reverse=True)
    
    print("Top 10 Layer-Position combinations:")
    print("Layer | Pos | Accuracy | Std Dev")
    print("-" * 35)
    for layer_idx, pos, acc, std in best_results[:10]:
        pos_str = "last" if pos == -1 else f"{pos:4d}"
        print(f" {layer_idx:2d}   | {pos_str} | {acc:.3f}   | {std:.3f}")
    
    print(f"\nRandom baseline for {len(unique_professions if attribute == 'profession' else unique_nationalities)}-way classification: {1/(len(unique_professions if attribute == 'profession' else unique_nationalities)):.3f}")
    print()

# Calculate position-averaged results (average over entity positions 3,4,5,6,7)
print("=== POSITION-AVERAGED RESULTS (Entity Positions vs Last Token) ===\n")

position_averaged_results = {
    'profession': {'entity_avg': {}, 'last_token': {}},
    'nationality': {'entity_avg': {}, 'last_token': {}}
}

for attribute in ['profession', 'nationality']:
    for layer_idx in probe_layer_indices:
        # Average over entity positions (3,4,5,6,7)
        entity_position_scores = []
        for pos in [3, 4, 5, 6, 7]:
            if pos in probe_summary[attribute][layer_idx]:
                entity_position_scores.extend(probe_summary[attribute][layer_idx][pos]['scores'])
        
        if entity_position_scores:
            position_averaged_results[attribute]['entity_avg'][layer_idx] = np.mean(entity_position_scores)
        
        # Last token position
        if -1 in probe_summary[attribute][layer_idx]:
            position_averaged_results[attribute]['last_token'][layer_idx] = \
                probe_summary[attribute][layer_idx][-1]['mean']

# Display the position-averaged comparison
for attribute in ['profession', 'nationality']:
    print(f"{attribute.upper()} - Entity Positions vs Last Token:")
    print("Layer | Entity Avg | Last Token | Difference")
    print("-" * 45)
    
    for layer_idx in probe_layer_indices:
        entity_avg = position_averaged_results[attribute]['entity_avg'].get(layer_idx, 0)
        last_token = position_averaged_results[attribute]['last_token'].get(layer_idx, 0)
        diff = last_token - entity_avg
        
        print(f" {layer_idx:2d}   |   {entity_avg:.3f}    |   {last_token:.3f}    | {diff:+.3f}")
    print()


In [ ]:
### Step 6: Create Comprehensive Visualizations

# Create multiple informative plots to understand the probe results
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Linear Probe Analysis: Attribute Representation Across Layers', fontsize=16, fontweight='bold')

# Plot 1: Profession probing - Entity positions vs Last token
ax1 = axes[0, 0]
entity_prof_layers = []
entity_prof_scores = []
last_prof_layers = []
last_prof_scores = []

for layer_idx in sorted(position_averaged_results['profession']['entity_avg'].keys()):
    if layer_idx in position_averaged_results['profession']['last_token']:
        entity_prof_layers.append(layer_idx)
        entity_prof_scores.append(position_averaged_results['profession']['entity_avg'][layer_idx])
        last_prof_layers.append(layer_idx)
        last_prof_scores.append(position_averaged_results['profession']['last_token'][layer_idx])

ax1.plot(entity_prof_layers, entity_prof_scores, 'o-', color='blue', linewidth=2, markersize=6, 
         label='Entity Positions (avg)', alpha=0.8)
ax1.plot(last_prof_layers, last_prof_scores, 's-', color='red', linewidth=2, markersize=6, 
         label='Last Token Position', alpha=0.8)

# Add random baseline
prof_baseline = 1/len(unique_professions)
ax1.axhline(y=prof_baseline, color='gray', linestyle='--', alpha=0.7, 
            label=f'Random Baseline ({prof_baseline:.3f})')

ax1.set_xlabel('Layer Index')
ax1.set_ylabel('Probe Accuracy')
ax1.set_title('Profession Classification Accuracy')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 1)

# Plot 2: Nationality probing - Entity positions vs Last token
ax2 = axes[0, 1]
entity_nat_layers = []
entity_nat_scores = []
last_nat_layers = []
last_nat_scores = []

for layer_idx in sorted(position_averaged_results['nationality']['entity_avg'].keys()):
    if layer_idx in position_averaged_results['nationality']['last_token']:
        entity_nat_layers.append(layer_idx)
        entity_nat_scores.append(position_averaged_results['nationality']['entity_avg'][layer_idx])
        last_nat_layers.append(layer_idx)
        last_nat_scores.append(position_averaged_results['nationality']['last_token'][layer_idx])

ax2.plot(entity_nat_layers, entity_nat_scores, 'o-', color='green', linewidth=2, markersize=6, 
         label='Entity Positions (avg)', alpha=0.8)
ax2.plot(last_nat_layers, last_nat_scores, 's-', color='orange', linewidth=2, markersize=6, 
         label='Last Token Position', alpha=0.8)

# Add random baseline
nat_baseline = 1/len(unique_nationalities)
ax2.axhline(y=nat_baseline, color='gray', linestyle='--', alpha=0.7, 
            label=f'Random Baseline ({nat_baseline:.3f})')

ax2.set_xlabel('Layer Index')
ax2.set_ylabel('Probe Accuracy')
ax2.set_title('Nationality Classification Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 1)

# Plot 3: Direct comparison of attributes at last token position
ax3 = axes[0, 2]
common_layers = sorted(set(last_prof_layers) & set(last_nat_layers))
prof_scores_common = [position_averaged_results['profession']['last_token'][l] for l in common_layers]
nat_scores_common = [position_averaged_results['nationality']['last_token'][l] for l in common_layers]

ax3.plot(common_layers, prof_scores_common, 'o-', color='red', linewidth=2, markersize=6, 
         label='Profession', alpha=0.8)
ax3.plot(common_layers, nat_scores_common, 's-', color='orange', linewidth=2, markersize=6, 
         label='Nationality', alpha=0.8)

ax3.axhline(y=prof_baseline, color='red', linestyle='--', alpha=0.5)
ax3.axhline(y=nat_baseline, color='orange', linestyle='--', alpha=0.5)

ax3.set_xlabel('Layer Index')
ax3.set_ylabel('Probe Accuracy')
ax3.set_title('Attribute Comparison (Last Token)')
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.set_ylim(0, 1)

# Plot 4: Heatmap of profession accuracy across layers and positions
ax4 = axes[1, 0]
# Create matrix for heatmap
positions_for_heatmap = [3, 4, 5, 6, 7, -1]  # Entity positions + last token
heatmap_data_prof = np.full((len(probe_layer_indices), len(positions_for_heatmap)), np.nan)

for i, layer_idx in enumerate(probe_layer_indices):
    for j, pos in enumerate(positions_for_heatmap):
        if pos in probe_summary['profession'][layer_idx]:
            heatmap_data_prof[i, j] = probe_summary['profession'][layer_idx][pos]['mean']

im1 = ax4.imshow(heatmap_data_prof, cmap='viridis', aspect='auto', vmin=0, vmax=1)
ax4.set_xticks(range(len(positions_for_heatmap)))
ax4.set_xticklabels(['Pos 3', 'Pos 4', 'Pos 5', 'Pos 6', 'Pos 7', 'Last'])
ax4.set_yticks(range(len(probe_layer_indices)))
ax4.set_yticklabels(probe_layer_indices)
ax4.set_xlabel('Token Position')
ax4.set_ylabel('Layer Index')
ax4.set_title('Profession Probe Accuracy Heatmap')
plt.colorbar(im1, ax=ax4, label='Accuracy')

# Plot 5: Heatmap of nationality accuracy across layers and positions
ax5 = axes[1, 1]
heatmap_data_nat = np.full((len(probe_layer_indices), len(positions_for_heatmap)), np.nan)

for i, layer_idx in enumerate(probe_layer_indices):
    for j, pos in enumerate(positions_for_heatmap):
        if pos in probe_summary['nationality'][layer_idx]:
            heatmap_data_nat[i, j] = probe_summary['nationality'][layer_idx][pos]['mean']

im2 = ax5.imshow(heatmap_data_nat, cmap='plasma', aspect='auto', vmin=0, vmax=1)
ax5.set_xticks(range(len(positions_for_heatmap)))
ax5.set_xticklabels(['Pos 3', 'Pos 4', 'Pos 5', 'Pos 6', 'Pos 7', 'Last'])
ax5.set_yticks(range(len(probe_layer_indices)))
ax5.set_yticklabels(probe_layer_indices)
ax5.set_xlabel('Token Position')
ax5.set_ylabel('Layer Index')
ax5.set_title('Nationality Probe Accuracy Heatmap')
plt.colorbar(im2, ax=ax5, label='Accuracy')

# Plot 6: Comparison with Patchscoping Results (Conceptual overlay)
ax6 = axes[1, 2]
# This plot shows how linear probe results relate to patchscoping findings
# We'll overlay the key insights from both analyses

ax6.plot(entity_prof_layers, entity_prof_scores, 'o-', color='blue', linewidth=2, markersize=6, 
         label='Profession: Entity Pos', alpha=0.8)
ax6.plot(last_prof_layers, last_prof_scores, 's-', color='red', linewidth=2, markersize=6, 
         label='Profession: Last Token', alpha=0.8)

# Add annotations for key findings
crossover_layer = 35  # From patchscoping analysis
ax6.axvline(x=crossover_layer, color='black', linestyle=':', alpha=0.7, 
            label='Patchscope Crossover (~Layer 35)')

ax6.set_xlabel('Layer Index')
ax6.set_ylabel('Probe Accuracy')
ax6.set_title('Linear Probes vs Patchscoping Insights')
ax6.legend()
ax6.grid(True, alpha=0.3)
ax6.set_ylim(0, 1)

plt.tight_layout()
plt.show()
